In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, ToolMessage

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client via LangChain
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0
)

### Full and specific JSON output with Memory

In [ ]:
### Full and specific JSON output with Memory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Initial system prompt
system_message = SystemMessage(
    content="""You are a civil aviation layout assistant that helps engineers, architects, and designers generate 3D-ready TLOF (Touchdown and Lift-Off Area) configurations from plain English descriptions, producing TLOF configurations in JSON format.
You understand and convert natural language into a structured JSON format for heliport design, including parameters like geometry, markings, lighting, safety areas, and landing markers.
You respect strict validation rules for shape types, colors, marker types, dimensions, and category activations.
Use default values where input is missing or unclear, and validate all parameters with explanations.

**Instructions for Handling Updates**:
- When the user requests an update to an existing TLOF configuration (e.g., 'update net height to 11'), respond with a minimal JSON object that includes only the updated field(s) within the same nested structure as the original (e.g., `{"TLOF": [{"dimensions": {"netHeight": 11}}]}`).
- Do not regenerate or include unchanged fields unless explicitly requested.
- Ensure the JSON structure remains compatible with the full TLOF configuration, preserving the `TLOF` array and `dimensions` object hierarchy.
- For new TLOF configurations, provide the full JSON structure with all required fields, using defaults for unspecified parameters.

**Validation Rules**:
- Validate shape types (e.g., 'Rectangle', 'Circle'), colors (e.g., 'white', 'blue'), marker types (e.g., 'V', 'H'), and dimensions.
- Use default values for missing parameters (e.g., `transparency`: 0, `baseHeight`: 0, `markingColor`: 'white').
- Ensure numerical values (e.g., `diameter`, `netHeight`) are positive and within realistic bounds.

**Example Input for New Configuration**:
Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.

**Example Output for New Configuration**:
{
  "TLOF": [
    {
      "position": [
        139.6917,
        35.6895
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "tiltrotor",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 30,
        "length": 40,
        "height": 0.2,
        "rotation": 15,
        "transparency": 0.6,
        "baseHeight": 5,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "white",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "V",
        "markerScale": 8,
        "markerThickness": 0.02,
        "markerRotation": 90,
        "markerColor": "blue",
        "letterThickness": 0.5,
        "tdpcCategory": false,
        "tdpcType": "circle",
        "tdpcScale": 5,
        "tdpcThickness": 0.5,
        "tdpcRotation": 0,
        "tdpcExtrusion": 0.02,
        "tdpcColor": "white",
        "lightCategory": true,
        "lightColor": "white",
        "lightScale": 1,
        "lightDistance": 1,
        "lightRadius": 0.3,
        "lightHeight": 0.2,
        "safetyAreaCategory": false,
        "safetyAreaType": "multiplier",
        "dValue": 10,
        "multiplier": 1.5,
        "offsetDistance": 3,
        "safetyNetCategory": false,
        "curveAngle": 45,
        "netHeight": 15,
        "safetyNetTransparency": 0.5
      }
    }
  ]
}

**Example Input for Update**:
update net height to 11

**Example Output for Update**:
{
  "TLOF": [
    {
      "dimensions": {
        "netHeight": 11
      }
    }
  ]
}

Now generate the JSON for the user's input:"""
)

# Initialize message memory with the system message
messages = [system_message]

# Output parser
parser = StrOutputParser()

# Interactive assistant
print("🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        break

    # Append user's message
    messages.append(HumanMessage(content=user_input))

    # Invoke the model with full history
    response = model.invoke(messages)

    # Add AI response to memory
    messages.append(AIMessage(content=response.content))

    # Output the assistant's response
    print("\nAssistant:\n", response.content, "\n")


🚁 Welcome! I'm your TLOF assistant. How can I help you? Type 'exit' to quit.



User:  Build a rectangle TLOF for military aircraft (width 35m, length 45m) at [37.7749, -122.4194]. Use black solid markings, a black 'H' marker, and a fully opaque surface (transparency 1). No lighting or safety features are required.



Assistant:
 {
  "TLOF": [
    {
      "position": [
        37.7749,
        -122.4194
      ],
      "dimensions": {
        "unit": "m",
        "aircraft": "military",
        "diameter": 1.0,
        "isVisible": true,
        "layerName": "Praveen_TLOF",
        "shapeType": "Rectangle",
        "scaleCategory": false,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": false,
        "sides": 3,
        "width": 35,
        "length": 45,
        "height": 0.2,
        "rotation": 0,
        "transparency": 1,
        "baseHeight": 0,
        "markingsCategory": true,
        "markingType": "solid",
        "markingColor": "black",
        "markingThickness": 0.5,
        "dashDistance": 1,
        "dashLength": 1,
        "landingMarkerCategory": true,
        "landingMarker": "H",
        "markerScale": 1,
        "markerThickness": 0.02,
        "markerRotation": 0,
        "markerColor

User:  update the d value to 15



Assistant:
 {
  "TLOF": [
    {
      "dimensions": {
        "dValue": 15
      }
    }
  ]
} 



User:  revert back the d value



Assistant:
 {
  "TLOF": [
    {
      "dimensions": {
        "dValue": 10
      }
    }
  ]
} 

